### Extract Aspect from three qualitative question from GenEd

In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import ast
from nltk import word_tokenize, pos_tag, pos_tag_sents

In [3]:
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords


from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet 
# Create WordNetLemmatizer object 


lemmatizer = WordNetLemmatizer() 

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jupyterlab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [184]:
df = pd.read_csv('Cleaned_CES_Pre_1st.csv')
second_df = pd.read_csv('Cleaned_CES_Final_1st.csv')
df = df.replace(np.nan, 'Neutral', regex=True) ### Some values are missing, use "Neutral" for missing values
df = df.drop(df.columns[[0]],axis=1) ### Drop extra index
test_df = df.iloc[0:, 0:] # Just for renaming purposes so that df variable can be used for testing
frame = [df,second_df]
test_df = pd.concat(frame)
test_df

,SUBJECT:,SECTION:,COURSE SCHEDULE,I can easily log-in and log-out my Canvas account.,I can access my course using the mobile Canvas app.,I can access my Canvas account using the available computers in JRU.,I have no issues accessing my Canvas account using JRU's wifi connection.,I have no issues accessing my Canvas account using the wired internet connection in the computer laboratories.,I can easily get help when I have problems with my Canvas account.,I can access my Canvas account outside JRU.,...,Canvas has made it easy for me to: Share ideas on group discussions,"I can answer the assessment materials (quiz, exam) posted in Canvas.",I find the course materials on Canvas aligned with the: Quizzes,"I find the course materials on Canvas aligned with the: Major (Prelim, Mideterm, Final) Exam",I find the course materials on Canvas aligned with the: Other tasks,I have been give enough time to accomplish the activities on Canvas.,What I like best in Canvas is/are:,What I like least in Canvas is/are:,I would like to suggest:,Unnamed: 0
0,HIS C101- Readings in Philippine History,BSA 101A,12:00 PM,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neutral,Neutral,Neutral,NaN
1,HIS C101- Readings in Philippine History,103R,12:18 PM,Disagree,Disagree,Disagree,Agree,Disagree,Agree,Agree,...,Disagree,Agree,Agree,Agree,Agree,Disagree,Neutral,Neutral,Neutral,NaN
2,HIS C101- Readings in Philippine History,101K,01:30 PM,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,Disagree,Agree,Agree,Strongly Agree,Agree,Strongly Agree,I can easily answer my quizzes and other forum...,Some announcement are not showing up in my not...,Create a power point moduel not a word and sum...,NaN
3,HIS C101- Readings in Philippine History,102-F,12:00 PM,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,Agree,Agree,Agree,Agree,Agree,Agree,It can di at home,Some of the instructions are not fully underst...,To be more active at face to face and lessen t...,NaN
4,HIS C101- Readings in Philippine History,101M,10:30 AM,Strongly Agree,Agree,Agree,Agree,Agree,Agree,Agree,...,Agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,give me some extra grades in specific course.,Neutral,Neutral,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,HIS C101- Readings in Philippine History,101LM,12:00 PM,Agree,Disagree,Agree,Agree,Agree,Agree,Agree,...,Disagree,Agree,Agree,Agree,Agree,Agree,NaN,NaN,NaN,2035.0
532,HIS C101- Readings in Philippine History,101 legal management,12:00 PM,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,NaN,NaN,NaN,2037.0
533,HIS C101- Readings in Philippine History,BSTM 1-1,9:00 AM,Strongly Agree,Strongly Agree,Agree,Agree,Agree,Agree,Strongly Agree,...,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Agree,NaN,NaN,NaN,2075.0
534,HIS C101- Readings in Philippine History,101 P,9:00 AM,Strongly Agree,Strongly Agree,Agree,Agree,Agree,Agree,Agree,...,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,NaN,NaN,NaN,2082.0


In [185]:
test_df = test_df.iloc[0:, -4:-1]
test_df = test_df.replace(np.nan, 'Neutral', regex=True)
test_df

,What I like best in Canvas is/are:,What I like least in Canvas is/are:,I would like to suggest:
0,Neutral,Neutral,Neutral
1,Neutral,Neutral,Neutral
2,I can easily answer my quizzes and other forum...,Some announcement are not showing up in my not...,Create a power point moduel not a word and sum...
3,It can di at home,Some of the instructions are not fully underst...,To be more active at face to face and lessen t...
4,give me some extra grades in specific course.,Neutral,Neutral
...,...,...,...
531,Neutral,Neutral,Neutral
532,Neutral,Neutral,Neutral
533,Neutral,Neutral,Neutral
534,Neutral,Neutral,Neutral


In [72]:
stoplist = stopwords.words('english') + ['though']
def remove_stopWords(w): 
    w = ' '.join(word for word in w.split() if word not in stoplist)
    return w

In [73]:
## Reduce list dimension

def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [116]:
col_range = len(test_df.columns) # number of columns
test_df = test_df.replace(r'^\s*$', "neutral", regex=True) ## If row value is null, replace with neutral string
aspect_list = []
aspect_words = []
for i in range(0,1):
    col = test_df.columns[i] # The current column
    test_df.loc[:,col] = test_df[col].apply(lambda x : str.lower(str(x))) ## To Lower Case
    test_df.loc[:,col] = test_df[col].apply(lambda x : " ".join(re.findall('[\w]+',x))) # Remove Punctuations
    test_df.loc[:,col] = test_df[col].apply(lambda x : remove_stopWords(x)) # Remove Stop words
    #test_df.loc[:,col] = test_df.loc[:,col].tolist()
    test_df.loc[:,col] = test_df[col].apply(lambda x : pos_tag_sents(map(word_tokenize, ast.literal_eval(x)))) # To list
    
    
    
    ##POS TAGGING
    #texts = test_df.loc[:,col].tolist()
    #tagged_texts = pos_tag_sents(map(word_tokenize, texts)) ### Tag every word in a row with POS
    
    
    def aspected(aspect):
        flattened_tagged=ast.literal_eval(aspect)
        flattened_tagged = flatten_list(aspect)
        prevWord=''
        prevTag=''
        currWord=''
        aspectList=[]
        outputDict={}

        #Extracting Aspects

        for word,tag in flattened_tagged:
            if(tag=='NN' or tag=='NNP'):
                if(prevTag=='NN' or prevTag=='NNP'):
                    currWord= prevWord + ' ' + word
                    return currWord

                else:
                    aspectList.append(prevWord.upper())
                    currWord= word

            prevWord=currWord
            prevTag=tag
            
    #test_df.loc[:,col] = test_df[col].apply(lambda x : aspected(x))
        
    #Eliminating aspect which has 1 or less count
    #aspect_words.append(aspectList)
    '''for aspect in aspectList:
            if(aspectList.count(aspect)>1):
                    if(outputDict.keys()!=aspect):
                            outputDict[aspect]=aspectList.count(aspect)
    outputAspect=sorted(outputDict.items(), key=lambda x: x[1],reverse = True)
    aspect_list.append(outputAspect)'''
    #print(outputAspect)
    

In [120]:
dd = pd.DataFrame(tagged[0])
dd['First'] = tagged[0]
dd['Second'] = tagged[1]
dd['Third'] = tagged[2]
dd = dd.iloc[0:, -3:]
dd.to_csv('pos_tags_opinion.csv',index=False)

In [169]:
ndf = pd.read_csv('pos_tags_opinion.csv')
ndf

,First,Second,Third
0,"[('neutral', 'JJ')]","[('neutral', 'JJ')]","[('neutral', 'JJ')]"
1,"[('neutral', 'JJ')]","[('neutral', 'JJ')]","[('neutral', 'JJ')]"
2,"[('easily', 'RB'), ('answer', 'NN'), ('quizzes...","[('announcement', 'NN'), ('showing', 'VBG'), (...","[('create', 'NN'), ('power', 'NN'), ('point', ..."
3,"[('di', 'NN'), ('home', 'NN')]","[('instructions', 'NNS'), ('fully', 'RB'), ('u...","[('active', 'JJ'), ('face', 'NN'), ('face', 'N..."
4,"[('give', 'VB'), ('extra', 'JJ'), ('grades', '...","[('neutral', 'JJ')]","[('neutral', 'JJ')]"
...,...,...,...
1062,"[('neutral', 'JJ')]","[('neutral', 'JJ')]","[('neutral', 'JJ')]"
1063,"[('neutral', 'JJ')]","[('neutral', 'JJ')]","[('neutral', 'JJ')]"
1064,"[('neutral', 'JJ')]","[('neutral', 'JJ')]","[('neutral', 'JJ')]"
1065,"[('neutral', 'JJ')]","[('neutral', 'JJ')]","[('neutral', 'JJ')]"


In [173]:
ndf = pd.read_csv('pos_tags_opinion.csv')
import ast

def get_aspect(aspect):
    
    aspectList = []
    prevWord=''
    prevTag=''
    currWord=''
    aspected= ''
    aspect = ast.literal_eval(aspect)
    for word,tag in aspect:
            if(tag=='NN' or tag=='NNP'):
                if(prevTag=='NN' or prevTag=='NNP'):
                    currWord= prevWord + ' ' + word
                    #return currWord
                    
                else:
                    aspectList.append(prevWord.upper())
                    currWord= word
                    
            
            prevWord=
            prevTag=tag
            return prevWord

In [174]:
for i in range(0,3):
    col = ndf.columns[i] # The current column
    
    texts = ndf.loc[:,col].tolist()
    tagged_texts = pos_tag_sents(map(word_tokenize, texts)) 
    
    ndf.loc[:,col] = ndf[col].apply(lambda x : get_aspect(x)) 


In [175]:
ndf

,First,Second,Third
0,,,
1,,,
2,,announcement,create
3,di,,
4,,,
...,...,...,...
1062,,,
1063,,,
1064,,,
1065,,,


In [188]:
df = pd.read_csv('Cleaned_CES_Pre_1st.csv')
second_df = pd.read_csv('Cleaned_CES_Final_1st.csv')
df = df.replace(np.nan, 'Neutral', regex=True) ### Some values are missing, use "Neutral" for missing values
df = df.drop(df.columns[[0]],axis=1) ### Drop extra index
test_df = df.iloc[0:, 0:] # Just for renaming purposes so that df variable can be used for testing
frame = [df,second_df]
test_df = pd.concat(frame)
test_df

test_df = test_df.iloc[0:, -4:-1]
test_df = test_df.replace(np.nan, 'Neutral', regex=True)
test_df

,What I like best in Canvas is/are:,What I like least in Canvas is/are:,I would like to suggest:
0,Neutral,Neutral,Neutral
1,Neutral,Neutral,Neutral
2,I can easily answer my quizzes and other forum...,Some announcement are not showing up in my not...,Create a power point moduel not a word and sum...
3,It can di at home,Some of the instructions are not fully underst...,To be more active at face to face and lessen t...
4,give me some extra grades in specific course.,Neutral,Neutral
...,...,...,...
531,Neutral,Neutral,Neutral
532,Neutral,Neutral,Neutral
533,Neutral,Neutral,Neutral
534,Neutral,Neutral,Neutral


In [189]:
col_range = len(test_df.columns) # number of columns
test_df = test_df.replace(r'^\s*$', "neutral", regex=True) ## If row value is null, replace with neutral string
aspect_list = []
aspect_words = []
tagged = []
for i in range(0,3):
    col = test_df.columns[i] # The current column
    test_df.loc[:,col] = test_df[col].apply(lambda x : str.lower(str(x))) ## To Lower Case
    test_df.loc[:,col] = test_df[col].apply(lambda x : " ".join(re.findall('[\w]+',x))) # Remove Punctuations
    test_df.loc[:,col] = test_df[col].apply(lambda x : remove_stopWords(x)) # Remove Stop words
    #test_df.loc[:,col] = test_df.loc[:,col].tolist()
    #test_df.loc[:,col] = test_df[col].apply(lambda x : pos_tag_sents(map(word_tokenize, ast.literal_eval(x)))) # To list
    
    
    
    ##POS TAGGING
    texts = test_df.loc[:,col].tolist()
    tagged_texts = pos_tag_sents(map(word_tokenize, texts)) ### Tag every word in a row with POS
    
    

    
    flattened_tagged = flatten_list(tagged_texts)
    prevWord=''
    prevTag=''
    currWord=''
    aspectList=[]
    outputDict={}

    #Extracting Aspects

    for word,tag in flattened_tagged:
        if(tag=='NN' or tag=='NNP'):
            if(prevTag=='NN' or prevTag=='NNP'):
                currWord= prevWord + ' ' + word
                

            else:
                aspectList.append(prevWord.upper())
                currWord= word
                
                
        prevWord=currWord
        prevTag=tag
        
        test_df.loc[:,col] = prevWord
    #test_df.loc[:,col] = test_df[col].apply(lambda x : aspected(x))
        
    #Eliminating aspect which has 1 or less count
    #aspect_words.append(aspectList)
    '''for aspect in aspectList:
            if(aspectList.count(aspect)>1):
                    if(outputDict.keys()!=aspect):
                            outputDict[aspect]=aspectList.count(aspect)
    outputAspect=sorted(outputDict.items(), key=lambda x: x[1],reverse = True)
    aspect_list.append(outputAspect)'''
    #print(outputAspect)
    

In [190]:
test_df

,What I like best in Canvas is/are:,What I like least in Canvas is/are:,I would like to suggest:
0,access,list announcement none,list nothing
1,access,list announcement none,list nothing
2,access,list announcement none,list nothing
3,access,list announcement none,list nothing
4,access,list announcement none,list nothing
...,...,...,...
531,access,list announcement none,list nothing
532,access,list announcement none,list nothing
533,access,list announcement none,list nothing
534,access,list announcement none,list nothing
